In [2]:
# Imports first
import csv
import time
from math import sqrt
#import geopandas as gpd
import pandas as pd
import shapely
from shapely.ops import nearest_points
import numpy as np
from scipy import ndimage
from scipy.spatial import cKDTree  
import pyproj

import matplotlib
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib.colors import ListedColormap
from matplotlib.ticker import MaxNLocator

In [3]:
categories: dict = {"THEFT": "property",
                    "BURGLARY": "property",
                    "MOTOR VEHICLE THEFT": "property",
                    "ARSON": "property",
                    "CRIMINAL DAMAGE": "property",
                    "ROBBERY": "property",
                    "ASSAULT": "person",
                    "BATTERY": "person",
                    "CRIM SEXUAL ASSAULT": "person",
                    "HOMICIDE": "person",
                    "INTIMIDATION": "person",
                    "KIDNAPPING": "person",
                    "OFFENSE INVOLVING CHILDREN": "person",
                    "SEX OFFENSE": "person",
                    "STALKING": "person",
                    "GAMBLING": "vice",
                    "NARCOTICS": "vice",
                    "PROSTITUTION": "vice",
                    "LIQUOR LAW VIOLATION": "vice",
                    "OBSCENITY": "vice",
                    "OTHER NARCOTIC VIOLATION": "vice",
                    "PUBLIC INDECENCY": "vice",
                    "OTHER OFFENSE": "other",
                    "DECEPTIVE PRACTICE": "other",
                    "WEAPONS VIOLATION": "other",
                    "PUBLIC PEACE VIOLATION": "other",
                    "CRIMINAL TRESPASS": "other",
                    "INTERFERENCE WITH PUBLIC OFFICER": "other",
                    "NON-CRIMINAL": "other"
                   }

In [4]:
category_feature_map = {'property': 1,
                        'person': 2,
                        'vice': 3,
                        'other': 4
                       }

In [5]:
feature_map: dict = {"THEFT": 1,
                    "BURGLARY": 2,
                    "MOTOR VEHICLE THEFT": 3,
                    "ARSON": 4,
                    "CRIMINAL DAMAGE": 5,
                    "ROBBERY": 6,
                    "ASSAULT": 7,
                    "BATTERY": 8,
                    "CRIM SEXUAL ASSAULT": 9,
                    "HOMICIDE": 10,
                    "INTIMIDATION": 11,
                    "KIDNAPPING": 12,
                    "OFFENSE INVOLVING CHILDREN": 13,
                    "SEX OFFENSE": 14,
                    "STALKING": 15,
                    "GAMBLING": 16,
                    "NARCOTICS": 17,
                    "PROSTITUTION": 18,
                    "LIQUOR LAW VIOLATION": 19,
                    "OBSCENITY": 20,
                    "OTHER NARCOTIC VIOLATION": 21,
                    "PUBLIC INDECENCY": 22,
                    "OTHER OFFENSE": 23,
                    "DECEPTIVE PRACTICE": 24,
                    "WEAPONS VIOLATION": 25,
                    "PUBLIC PEACE VIOLATION": 26,
                    "CRIMINAL TRESPASS": 27,
                    "INTERFERENCE WITH PUBLIC OFFICER": 28,
                    "NON-CRIMINAL": 29
                   }

In [6]:
crimes = pd.read_pickle('data/crimes-transformed.pkl')

In [7]:
crimes.head()

,ID,case number,date,block,iucr,primary type,desc,locdesc,arrest,domestic,...,y coord,year,updated on,lat,lon,location,community,UTMx,UTMy,UTMPoint
0,8265244,HT498458,09/15/2011 08:00:00 AM,040XX N MENARD AVE,0620,BURGLARY,UNLAWFUL ENTRY,RESIDENCE,True,False,...,1926315.0,2011,02/12/2018 03:46:59 PM,41.953965,-87.771918,"(41.95396528, -87.771918163)",PORTAGE PARK,436025.308686,4.644953e+06,POINT (436025.3086861699 4644953.17679981)
1,7357772,HS159408,07/01/2001 10:00:00 AM,056XX W CORNELIA AVE,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,...,1922746.0,2001,02/18/2010 01:12:55 AM,41.944150,-87.767546,"(41.9441497, -87.767545572)",PORTAGE PARK,436377.940882,4.643860e+06,POINT (436377.940881997 4643860.108357577)
2,7609828,HS413731,07/13/2010 12:00:00 PM,041XX N LONG AVE,0820,THEFT,$500 AND UNDER,PARK PROPERTY,False,False,...,1926816.0,2010,02/04/2016 06:33:39 AM,41.955292,-87.762142,"(41.95529191, -87.762141971)",PORTAGE PARK,436836.850742,4.645093e+06,POINT (436836.8507415869 4645093.21940705)
3,7610439,HS413034,07/16/2010 03:00:00 AM,056XX W SCHOOL ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,1921375.0,2010,02/04/2016 06:33:39 AM,41.940387,-87.767568,"(41.940387482, -87.767567805)",PORTAGE PARK,436372.357517,4.643442e+06,POINT (436372.3575166143 4643442.41025281)
4,7610466,HS414753,07/17/2010 12:25:00 AM,032XX N CICERO AVE,1506,PROSTITUTION,SOLICIT ON PUBLIC WAY,ALLEY,True,False,...,1920986.0,2010,02/04/2016 06:33:39 AM,41.939216,-87.746878,"(41.939216164, -87.746877814)",PORTAGE PARK,438086.334871,4.643297e+06,POINT (438086.3348712226 4643297.20981963)


In [8]:
crimes['primary type id'] = crimes['primary type'].apply(lambda x: feature_map[x])

In [9]:
crimes['primary type category'] = crimes['primary type'].apply(lambda x: categories[x])

In [10]:
crimes['primary type category id'] = crimes['primary type category'].apply(lambda x: category_feature_map[x])

In [11]:
communities = pd.read_pickle('data/neighborhoods-transformed.pkl')

In [12]:
communities[communities['community'] == crimes.iloc[0]['community']]

,community,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len,geometry
14,PORTAGE PARK,0,110196097.139,0,15,15.0,0,0,46520.6421377,(POLYGON ((-87.75263506823083 41.9679674665160...


In [16]:
crimes.to_pickle('data/crimes-transformed.pkl')

In [18]:
crimes.head()

,ID,case number,date,block,iucr,primary type,desc,locdesc,arrest,domestic,...,lat,lon,location,community,UTMx,UTMy,UTMPoint,primary type id,primary type category,primary type category id
0,8265244,HT498458,09/15/2011 08:00:00 AM,040XX N MENARD AVE,0620,BURGLARY,UNLAWFUL ENTRY,RESIDENCE,True,False,...,41.953965,-87.771918,"(41.95396528, -87.771918163)",PORTAGE PARK,436025.308686,4.644953e+06,POINT (436025.3086861699 4644953.17679981),2,property,1
1,7357772,HS159408,07/01/2001 10:00:00 AM,056XX W CORNELIA AVE,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,...,41.944150,-87.767546,"(41.9441497, -87.767545572)",PORTAGE PARK,436377.940882,4.643860e+06,POINT (436377.940881997 4643860.108357577),1,property,1
2,7609828,HS413731,07/13/2010 12:00:00 PM,041XX N LONG AVE,0820,THEFT,$500 AND UNDER,PARK PROPERTY,False,False,...,41.955292,-87.762142,"(41.95529191, -87.762141971)",PORTAGE PARK,436836.850742,4.645093e+06,POINT (436836.8507415869 4645093.21940705),1,property,1
3,7610439,HS413034,07/16/2010 03:00:00 AM,056XX W SCHOOL ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,41.940387,-87.767568,"(41.940387482, -87.767567805)",PORTAGE PARK,436372.357517,4.643442e+06,POINT (436372.3575166143 4643442.41025281),8,person,2
4,7610466,HS414753,07/17/2010 12:25:00 AM,032XX N CICERO AVE,1506,PROSTITUTION,SOLICIT ON PUBLIC WAY,ALLEY,True,False,...,41.939216,-87.746878,"(41.939216164, -87.746877814)",PORTAGE PARK,438086.334871,4.643297e+06,POINT (438086.3348712226 4643297.20981963),18,vice,3
